# Goal data

## Imports

In [ ]:
# !pip install meteostat
# !pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
import random
import json
from math import sqrt

from statsmodels.graphics.tsaplots import plot_acf

import tensorflow_probability as tfp
from scipy.stats import gennorm
import scipy.stats
import scipy.stats as stats
from scipy.stats import johnsonsu
from scipy.stats import exponnorm
from sklearn.metrics import mutual_info_score
from sklearn.metrics import mean_squared_error
from scipy.stats import mstats

from prophet import Prophet
import itertools
from prophet.diagnostics import performance_metrics
from prophet.diagnostics import cross_validation
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.serialize import model_to_json, model_from_json

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly

from sklearn.preprocessing import StandardScaler
from tensorflow import expand_dims
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, Bidirectional, Lambda
from tensorflow.keras.callbacks import EarlyStopping

import holidays
import joblib

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

## Functions

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
def plot_actual_forecasted(actual, predicted):
    '''
    predicted, actual: pd.Series. actual should have timestamp index
    '''
    plt.plot(actual.index, actual.values, label='Actual')
    plt.plot(actual.index, predicted.values, label='Forecasted')
    plt.legend()

In [ ]:
def smooth_data(df, basis="D", func='mean'):
    '''
    averages data according to some rule and returns new dataframe
    params:
    df: pd.DataFrame, dataframe to smooth data
    basis: str, on which basis to average a data, can be "D", "W", "M", "Y"
    '''
    if func == 'mean':
        return df.resample(rule=basis).mean()
    if func == 'sum':
        return df.resample(rule=basis).sum()

In [ ]:
def set_datetime_index(df, date_name):
    '''
    df: pd.DataFrame
    date_name: str, name of column that should be date time index
    returns dataFrame with new index
    '''
    df = df.set_index(date_name)
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
def mutual_info_with_df(other_df, col_other, price_df=None):
    try:
        price_df=gen_rt_df
    except NameError:
        return "Please read generator data first"
    for i in ['M', 'W', 'D', 'H', '30min', '5min']:
        print('\n\n ', i, ':')
        temp_other = smooth_data(other_df, i)[col_other].dropna()
        temp_gen = smooth_data(price_df, i).dropna()
        temp_df = temp_gen.merge(temp_other, how='inner', left_index=True, right_index=True)
        # print('LBMP: ', mutual_info_score(temp_other['Load'], temp_gen['LBMP ($/MWHr)']))
        print('Marginal price: ', mutual_info_score(temp_df[col_other], temp_df['Marginal energy']))

In [ ]:
def get_param_grid(params_grid):
    '''
    params_grid: dictionary of parameters of the next structure:
    params_grid = {
    'growth': ['linear', 'logistic'],
    'changepoint_prior_scale': [0.01, 0.1, 1.0],
    }
    Returns list of combinations of these parameters, like
    [{'growth': 'logistic',
    'changepoint_prior_scale': 0.01}, {...}]
    '''
    param_values = list(params_grid.values())
    param_names = list(params_grid.keys())

    param_combinations = itertools.product(*param_values)

    param_grid_list = []
    for combination in param_combinations:
        param_dict = {}
        for i in range(len(param_names)):
            param_dict[param_names[i]] = combination[i]
        param_grid_list.append(param_dict)

    return param_grid_list


In [ ]:
def plot_corr_plot(df_corr, figsize=(10,8), cmap="coolwarm"):
    '''
    df_corr: pd.DataFrame created by function pd.DF.corr()
    '''
    mask = np.zeros_like(df_corr, dtype=bool)
    mask[np.triu_indices_from(mask)] = True
    df_corr[mask] = np.nan

    plt.figure(figsize=figsize)
    g = sns.heatmap(df_corr, cmap="coolwarm", annot=True)
    g.set_xticklabels(g.get_xticklabels(),
                      rotation = 60,
                      # fontsize = 8
                      )
    plt.show()

In [ ]:
def plot_autocorr(data, lags=100, figsize=(15, 10)):
    '''
    Plots autocorrelation function
    data: pd.Series
    '''
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(1, 1, 1)
    plot_acf(x=data, lags=lags, ax=ax)

## Variables

In [ ]:
lbmp = 'LBMP ($/MWHr)'
start_date='2022-01-01'
end_date = '2023-03-16'
smoothing_param = '3H'
chosen_generator = 'ADK S GLENS___FALLS'

train_percentile = 0.8
valid_percentile = 0.975



## Generator real time LBPM

### Data reading

In [ ]:
path_ = '/content/gdrive/MyDrive/Diploma/Data/real_time_generator_one_csv/real_time_generator_pivoted.csv'
gen_rt_df = set_datetime_index(pd.read_csv(path_), 'Time Stamp')

# removing generators where more than 50% of nans
na_percentages = gen_rt_df.isna().sum() / len(gen_rt_df)
cols_to_remove = na_percentages[na_percentages > 0.5].index.tolist()
gen_rt_df.drop(cols_to_remove, axis=1, inplace=True)

gen_rt_df = gen_rt_df.loc[start_date:end_date]


### Correlations of LBMPs

In [ ]:
corr_df = gen_rt_df.iloc.corr()

# plt.figure(figsize=(110,80))
# g = sns.heatmap(corr_df, cmap="coolwarm", annot=True)
# g.set_xticklabels(g.get_xticklabels(),
#                   rotation = 60,
#                   # fontsize = 8
#                   )
# plt.show()

In [ ]:
corr_df.describe().loc['mean'].mean()

So, in average, correlation of lbmps is 0.74, which is high. And so, choosing features for one is benefecial for all of them.

In [ ]:
corr_df['ADK S GLENS___FALLS'].mean()

mean correlation of my generator is 0.69, which is quite close to average by dataset, so it should be nice.

### Overall analysis

#### Overall distribution

In [ ]:
plt.hist(gen_rt_df.values.flatten(), bins='auto', edgecolor='blue')

plt.xlabel('LBMP')
plt.ylabel('Frequency')
plt.xlim(-50, 500)
plt.show()

#### Outliers removal

In [ ]:
# removing outliers with IQR
for gen in gen_rt_df.columns:
    sr = pd.DataFrame(gen_rt_df[gen])
    quantile25 = sr.quantile(.25)
    quantile75 = sr.quantile(.75)
    iqr = quantile75 - quantile25
    upper = float(quantile75 + 1.5 * iqr)
    lower = float(quantile25 - 1.5 * iqr)
    sr.loc[(sr[gen] < lower) | (sr[gen] > upper), gen] = np.nan
    sr[gen] = sr[gen].fillna(method='ffill')
    sr[gen] = sr[gen].fillna(method='bfill')
    gen_rt_df[gen] = sr[gen]

gen_rt_df.head()

#### Visualising random timestamp distribution

In [ ]:
grp = gen_rt_df.groupby(pd.Grouper(freq='3H')).apply(lambda x: x.groupby(by='Name').mean())
grp

In [ ]:
grp = grp.groupby(pd.Grouper(freq='3H', level='Time Stamp'))


In [ ]:
t = list(grp.groups)
t[:5]

In [ ]:
n = 0

row_num = 3
col_num = 6
    
def visualise_distributions():
    fig, axs = plt.subplots(row_num, col_num, figsize=(17, 10))
    fig.suptitle("Data distributions")
    
    for row in range(row_num):
        for col in range(col_num):
            # if row == row_num-1 and col == col_num-1:
            #     break
            time_chosen = random.choice(t)
            data_grouped = grp.get_group(time_chosen)
            data_grouped = data_grouped.groupby('Name').mean()
    
            data_grouped = data_grouped[data_grouped[lbmp] < data_grouped[lbmp].quantile(0.99)]
            data_grouped = data_grouped[data_grouped[lbmp] > data_grouped[lbmp].quantile(0.01)]
            
            axs[row, col].hist(data_grouped[lbmp], bins='auto' )#density=True)
            axs[row, col].set_title(time_chosen)
            # index += 1
    
visualise_distributions()    
    

# All other data

## Real Time Actual Load

In [ ]:
pathh = '/content/gdrive/MyDrive/Diploma/Data/real_time_actual_load_one_csv/real_time_actual_load.csv'
load_rt_df_sum = pd.read_csv(pathh)
load_rt_df_sum = load_rt_df_sum.set_index('Time Stamp')
load_rt_df_sum.index = pd.to_datetime(load_rt_df_sum.index)
load_rt_df_sum.head()

### ACF

In [ ]:
load_rt_df_sum.head()

In [ ]:
print(type(load_rt_df_sum))
print(type(temp_df))
load_rt_df_sum.values

In [ ]:
temp_df = smooth_data(load_rt_df_sum, smoothing_param)
# plot_autocorr(temp_df['Load'], lags=50)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plot_acf(x=temp_df, lags=list(range(50)), ax=ax)

In [ ]:
load_rt_df_sum

In [ ]:
tmp = smooth_data(load_rt_df_sum, 'H').dropna()

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(1, 1, 1)
plot_acf(x=tmp['Load'], lags=list(range(100)), ax=ax)

## Balancing Market Advisory
Bids, schedules, etc. seems like actual

In [ ]:
def read_balancing_market(time_stamps='All'):
    '''
    time_stamps: int, number of files to read. Can be "All"
    '''
    path = '/content/gdrive/MyDrive/Diploma/Data/BalancingMarketAdvisory/'

    filenames = []

    for entry in os.listdir(path):
        if os.path.isfile(os.path.join(path, entry)):
            filenames.append(entry)
    filenames.sort(reverse=True)

    # market_rt_df = pd.DataFrame(columns=['Start Time', 'Name', 'PTID', 'LBMP ($/MWHr)',
    #                                   'Marginal Cost Losses ($/MWHr)', 'Marginal Cost Congestion ($/MWHr)']).set_index('Start Time')
    i = 0
    # TODO remove slice
    if time_stamps=='All':
        time_stamps = len(filenames)
    for filename in filenames[:time_stamps]:
        print(f'{i}/{len(filenames)}')
        i += 1
        with zipfile.ZipFile(path + filename) as zipobj:
            inside_csvs = zipobj.namelist()
            for inside_csv in inside_csvs:
                temp_market_rt_df = pd.read_csv(zipobj.open(inside_csv))

                ind = temp_market_rt_df[temp_market_rt_df['Start Time'] == 'Total'].index
                temp_market_rt_df = temp_market_rt_df.drop(ind)

                temp_market_rt_df = temp_market_rt_df.set_index('Start Time')

                try:
                    market_rt_df = pd.concat([market_rt_df, temp_market_rt_df])
                except NameError:
                    market_rt_df = temp_market_rt_df

    market_rt_df.index = pd.to_datetime(market_rt_df.index)

    # TODO remove duplicates?
    # gen_rt_df['Marginal energy'] = gen_rt_df['LBMP ($/MWHr)'] - gen_rt_df['Marginal Cost Losses ($/MWHr)'] + gen_rt_df['Marginal Cost Congestion ($/MWHr)']
    market_rt_df = market_rt_df.sort_values('Start Time')

    market_rt_df = market_rt_df[~market_rt_df.index.duplicated(keep='first')]

    market_rt_df = market_rt_df.loc[start_date:end_date]

    return market_rt_df

In [ ]:
market_rt_dff = read_balancing_market()
market_rt_dff

things to include: Generation Scheduled (0.25), Wheel Throughs Bid (0.25), Gross Imports NYISO (0.25). Maybe Gross Exports HQ (0.21), Gross Exports PJM (0.20)




In [ ]:
# market_rt_dff[lbmp] = smooth_data(gen_rt_df[lbmp], 'H')
# market_rt_dff.corr()[lbmp]

#### ACF

In [ ]:
plot_autocorr(market_rt_dff['Wheel Throughs Bid'], lags=100, figsize=(8, 5))

In [ ]:
# TODO pay attention: just take generation capacity bid and generation scheduled as only needed info. I also will have a price, so it can be estimated.

In [ ]:
# TODO calculate how this info is related to 

In [ ]:
plot_autocorr(market_rt_dff['Gross Exports HQ'], lags=100, figsize=(8, 5))

In [ ]:
plot_autocorr(market_rt_dff['Gross Exports PJM'], lags=100, figsize=(8, 5))

In [ ]:
plot_autocorr(market_rt_dff['Gross Imports NYISO'], lags=100, figsize=(8, 5))


In [ ]:
plot_autocorr(market_rt_dff['Generation Scheduled'], lags=100, figsize=(8, 5))


## Natural gas

In [ ]:
path_gas = "/content/gdrive/MyDrive/Diploma/Data/natural_gas/Gas_Historical_Data.csv"
gas_df = pd.read_csv(path_gas)
gas_df = gas_df.set_index("Date")
gas_df.index = pd.to_datetime(gas_df.index)
gas_df = gas_df.sort_index(ascending=True)
gas_df = gas_df.loc[start_date:end_date]
gas_df.head()

In [ ]:
plt.plot(gas_df.index, gas_df['Close/Last'])

### ACF

In [ ]:
plot_autocorr(gas_df['Close/Last'], lags=200)

## Coal

In [ ]:
coal_df = pd.read_csv('/content/gdrive/MyDrive/Diploma/Data/Coal/Coal_03_17_23-10_16_06.csv')
coal_df = coal_df.set_index("Date")
coal_df.index = pd.to_datetime(coal_df.index)
coal_df = coal_df.loc[start_date:end_date]

coal_df.head()

In [ ]:
plt.plot(coal_df.index, coal_df['Close'])

## Weather data

In [ ]:
# works only from second try, for some reason

In [ ]:
lst_start = start_date.split('-')
lst_end = end_date.split('-')

start = datetime(int(lst_start[0]), int(lst_start[1]), int(lst_start[2]), hour=00)
end = datetime(int(lst_end[0]), int(lst_end[1]), int(lst_end[2])+1)

# Create Point for Vancouver, BC
new_york = Point(40.805877, -74.409130, 70)

# can be hourly
data = Hourly(new_york, start, end)
data = data.fetch()

# Plot line chart including average, minimum and maximum temperature
# data.plot(y=['prcp'], figsize=(30, 20))   #, 'tmin', 'tmax'])
# plt.show()

In [ ]:
data[['temp', 'pres', 'dwpt', 'rhum', 'prcp', 'wspd', 'pres']].corr()

## Crude Oil

In [ ]:
oil_df = pd.read_csv('/content/gdrive/MyDrive/Diploma/Data/Crude_Oil/Oil (WTI)_03_17_23-01_20_05.csv')
oil_df = oil_df.set_index("Date")
oil_df.index = pd.to_datetime(oil_df.index)
oil_df = oil_df.loc[start_date:end_date]
oil_df.head()

In [ ]:
plt.plot(oil_df.index, oil_df['Close'])

In [ ]:
for i in corr_types:
    print(i, ": ", oil_df['Close'].corr(smooth_data(df_main, "D")['DAM Gen LBMP'], method=i))

## Uranium

In [ ]:
u_df = pd.read_csv('/content/gdrive/MyDrive/Diploma/Data/Uranium/URA Historical Data 2010-2023.csv')
u_df = u_df.set_index("Date")
u_df.index = pd.to_datetime(u_df.index)
u_df = u_df.loc[start_date:end_date]
u_df.head()

In [ ]:
plt.plot(u_df.index, u_df['Price'])

# Models training

## Models preparation

In [ ]:
class GenericModel:

    def __init__(self):
        self.default_path = '/content/gdrive/MyDrive/Diploma/Models/'
        self.default_extention = '.pkl'
        return None

    def fit(self, train_x, train_y):
        raise NotImplementedError

    def predict(self, test_x):
        raise NotImplementedError

    def save(self, path):
        raise NotImplementedError

    def read_params(self, path):
        raise NotImplementedError

### Prophet

In [ ]:
class MyFbProphet(GenericModel):
    param_grid = {
                  'growth': ['linear'], #, 'logistic'],
                  'changepoint_prior_scale': [0.01, 0.1, 0.5,    0.001],
                  'seasonality_mode': ['additive', 'multiplicative'],
                  'seasonality_prior_scale': [0.1, 5.0,     1, 10],
                  # TODO del?
                  # 'changepoint_range': [0.8, 0.95],
                  # 'holidays_prior_scale': [0.01, 1, 10]
                  }
    name = 'prophet_model'

    def __init__(self, features, params=dict()):
        '''
        features: list of column names of train_x to add to model
        params: dictionary of parameters
        '''
        super(MyFbProphet, self).__init__()
        self.model = Prophet(**params, stan_backend=None)
        self.features = features

    def fit(self, train_x, train_y):
        '''
        
        '''
        for feature in self.features:
            self.model.add_regressor(feature)
        train_x['ds'] = train_x.index
        train_x['y'] = train_y.values
        self.model.fit(train_x[['ds', 'y'] + self.features])
        train_x.drop(['ds', 'y'], axis=1, inplace=True)

    def predict(self, test_x):
        test_x['ds'] = test_x.index
        result = self.model.predict(test_x[['ds'] + self.features])['yhat']
        test_x.drop(['ds'], axis=1, inplace=True)
        return result

    def save(self, name=None,  parameters=None):
        if not name:
            name = self.name
        work_path = self.default_path + name + self.default_extention
        joblib.dump(parameters, work_path)
        # with open(work_path, 'w') as fout:
        #     fout.write(model_to_json(self.model))

    def read_params(self, name=None):
        if not name:
            name = self.name
        work_path = self.default_path + name + self.default_extention
        params = joblib.load(work_path)
        self.model = Prophet(**params)
        # with open(work_path, 'r') as fin:
        #     self.model = model_from_json(fin.read())


### Gradient boosting

In [ ]:
class MyGradientBoosting(GenericModel):
    param_grid = {
                  'n_estimators': [100, 300],
                  'max_depth': [4, 7],
                  # 0.01 does not show good results.
                  'learning_rate': [0.1], #, 0.01],
                  'subsample': [0.8, 0.9],
                  'colsample_bytree': [0.8, 0.9]
                  }
    name = 'gradient_boosting_model'

    def __init__(self, features, params=dict()):
        super(MyGradientBoosting, self).__init__()
        self.model = XGBRegressor(**params)
        self.features = features
        

    def fit(self, train_x, train_y):
        self.model.fit(train_x[self.features], train_y)

    def predict(self, test_x):
        result = self.model.predict(test_x[self.features])
        return result

    def save(self, name=None, parameters=None):
        if not name:
            name = self.name
        if not parameters:
            parameters = self.model.get_params()
        work_path = self.default_path + name + self.default_extention
        joblib.dump(parameters, work_path)

    def read_params(self, name=None):
        if not name:
            name = self.name
        work_path = self.default_path + name + self.default_extention
        params = joblib.load(work_path)
        self.model = XGBRegressor(**params)

### Random Forest

In [ ]:
class MyRandomForest(GenericModel):
    param_grid = {
                  'n_estimators': [100, 300],
                  'max_depth': [4, 8],
                  'min_samples_split': [2, 10],
                  'min_samples_leaf': [1, 4],
                  'max_features': [1, 'sqrt']
                  }
    name = 'random_forest_model'

    def __init__(self, features, params=dict()):
        super(MyRandomForest, self).__init__()
        self.model = RandomForestRegressor(**params)
        self.features = features

    def fit(self, train_x, train_y):
        self.model.fit(train_x[self.features], train_y)

    def predict(self, test_x):
        result = self.model.predict(test_x[self.features])
        return result

    def save(self, name=None, parameters=None):
        if not name:
            name = self.name
        if not parameters:
            parameters = self.model.get_params()
        work_path = self.default_path + name + self.default_extention
        joblib.dump(self.model.get_params(), work_path)

    def read_params(self, name=None):
        if not name:
            name = self.name
        work_path = self.default_path + name + self.default_extention
        params = joblib.load(work_path)
        self.model = RandomForestRegressor(**params)

### Linear Regression

In [ ]:
class MyLinearRegression(GenericModel):
    param_grid = {
                  'fit_intercept': [True, False],
                  'n_jobs': [None, 100],
                  }
    name = 'linear_regression_model'

    def __init__(self, features, params=dict()):
        super(MyLinearRegression, self).__init__()
        self.model = LinearRegression()
        self.model.set_params(**params)
        self.features = features

    def fit(self, train_x, train_y):
        self.model.fit(train_x[self.features], train_y)

    def predict(self, test_x):
        result = self.model.predict(test_x[self.features])
        return result

    def save(self, name=None, parameters=None):
        if not name:
            name = self.name
        if not parameters:
            parameters = self.model.get_params()
        work_path = self.default_path + name + self.default_extention
        joblib.dump(self.model.get_params(), work_path)

    def read_params(self, name=None):
        if not name:
            name = self.name
        work_path = self.default_path + name + self.default_extention
        params = joblib.load(work_path)
        self.model = LinearRegression(**params)

### CNN

In [ ]:
class MyCNN(GenericModel):
    param_grid = {
                  'filters': [32, 64, 128],
                  'kernel_size': [3, 5],
                  'dense_layer_size': [50, 100]
                  }
    name = 'cnn_model'

    def __init__(self, features, params=dict()):
        super(MyCNN, self).__init__()
        
        if len(params) == 0:
            self.model = self.create_model()
        else:
            self.model = self.create_model(**params)     
        self.features = features

    def create_model(self, filters=64, kernel_size=3, dense_layer_size=50):
        mdl = Sequential()
        mdl.add(Conv1D(filters=filters,
                         kernel_size=kernel_size,
                         activation='relu',
                         input_shape=(len(features), 1)))
        mdl.add(MaxPooling1D(pool_size=2))
        mdl.add(Flatten())
        mdl.add(Dense(dense_layer_size, activation='relu'))
        mdl.add(Dense(1))
        mdl.compile(optimizer='adam',
                      loss='mean_absolute_percentage_error')
        return mdl

    def fit(self, train_x, train_y):
        train_x = train_x[self.features]
        train_x_cnn = np.reshape(train_x.values,
                                 (train_x.shape[0], train_x.shape[1], 1))
        # verbose: 0 - see nothing, 1 - see animation, 2 - see epoch count
        self.model.fit(train_x_cnn, train_y,
                       epochs=15, batch_size=16, verbose=0)

    def predict(self, test_x):
        test_x = test_x[self.features]
        test_x_cnn = np.reshape(test_x.values,
                                (test_x.shape[0], test_x.shape[1], 1))
        y_pred = self.model.predict(test_x_cnn)
        result = y_pred.reshape(-1)
        return result

    def save(self, name=None, parameters=None):
        if not name:
            name = self.name
        work_path = self.default_path + name + self.default_extention
        joblib.dump(parameters, work_path)
        # self.model.save(work_path)

    def read_params(self, name=None):
        if not name:
            name = self.name
        work_path = self.default_path + name + self.default_extention
        params = joblib.load(work_path)
        self.model = self.create_model(**params)

### LSTM (DEPRECATED)

In [ ]:
class MyLSTM(GenericModel):
    param_grid = {
                  'optimizer': ['adam'],
                  'neurons': [10, 50, 100],
                  'activation': ['relu', 'sigmoid'],
                  'dropout': [0.2, 0.4]
                  }
    name = 'lstm_model'

    def __init__(self, features, params=dict()):
        super(MyLSTM, self).__init__()
        
        if len(params) == 0:
            self.model = self.create_model()
        else:
            self.model = self.create_model(**params)     
        self.features = features

    def create_model(self, optimizer='adam', neurons=50, activation='relu', dropout=0.2):
        mdl = Sequential([ 
        Bidirectional(LSTM(units=neurons, input_shape=(1, len(features)),
                           activation=activation,
                           return_sequences=True)),
        Dropout(dropout),
        # Bidirectional(LSTM(units=neurons, return_sequences=True)),
        # Dropout(dropout),
        Bidirectional(LSTM(units=neurons, return_sequences=False)),
        Dropout(dropout),
        Dense(1),
        ])

        mdl.compile(optimizer=optimizer, loss='mean_absolute_percentage_error')
        return mdl

    def fit(self, train_x, train_y):
        self.scaler = StandardScaler()
        train_x_scaled = self.scaler.fit_transform(train_x)
        
        n_features = len(features) # train_x_scaled.shape[1]
        train_x_lstm = train_x_scaled.reshape(train_x_scaled.shape[0], 1, n_features)

        early_stop = EarlyStopping(monitor='loss', patience=5, mode='min')

        self.model.fit(train_x_lstm, train_y,
                       epochs=15,
                       batch_size=32,
                       verbose=0,
                       callbacks=[early_stop])

    def predict(self, test_x):
        test_x_scaled = self.scaler.transform(test_x)
        
        test_x_lstm = test_x_scaled.reshape(test_x_scaled.shape[0],
                                            1, len(features))
        result = self.model.predict(test_x_lstm)
        return result

    def save(self, name=None, parameters=None):
        if not name:
            name = self.name
        work_path = self.default_path + name + self.default_extention
        joblib.dump(parameters, work_path)
        # self.model.save(work_path)

    def read_params(self, name=None):
        if not name:
            name = self.name
        work_path = self.default_path + name + self.default_extention
        params = joblib.load(work_path)
        self.model = self.create_model(**params)
        # self.model = load_model(work_path)

## Dataset preparation

In [ ]:
# data_df = smooth_data(gen_rt_df, 'H')

data_df = pd.DataFrame(gen_rt_df[chosen_generator].rename(lbmp, inplace=True))

data_df['Load'] = smooth_data(load_rt_df_sum, 'H')
data_df = data_df[[lbmp, 'Load']]

data_df['Natural Gas'] = gas_df['Close/Last']
data_df['Coal'] = coal_df['Close']
data_df['Crude Oil'] = oil_df['Close']
data_df['Uranium'] = u_df['Price']

data_df['Temperature'] = data['temp']
data_df['Precip'] = data['prcp']
data_df['Precip']= data_df['Precip'].fillna(0)
data_df['Pressure'] = data['pres']
data_df['Wind speed'] = data['wspd']
data_df['Dew temperature'] = data['dwpt']
data_df['Relative humidity'] = data['rhum']

data_df['Gen Scheduled'] = market_rt_dff['Generation Scheduled']
data_df['Wheel Throughs Bid'] = market_rt_dff['Wheel Throughs Bid']
data_df['Imports NYISO'] = market_rt_dff['Gross Imports NYISO']
data_df['Exports HQ'] = market_rt_dff['Gross Exports HQ']
data_df['Exports PJM'] = market_rt_dff['Gross Exports PJM']

data_df = data_df.fillna(method='ffill')
data_df = data_df.fillna(method='bfill')

us_holidays = holidays.US()
data_df['Is holiday'] = pd.Series(data_df.index).apply(lambda x: 1 if x in us_holidays else 0).values

data_df = data_df.loc[start_date:end_date]
data_df = smooth_data(data_df, smoothing_param)

data_df.head()

In [ ]:
# features = ['Load', 'Natural Gas', 'Coal', 'Crude Oil', 'Uranium',
#             'Temperature', 'Precip', 'Pressure', 'Wind speed',
#             'Dew temperature', 'Relative humidity', 'Is holiday', 
#             'Gen Scheduled', 'Wheel Throughs Bid', 'Imports NYISO',
#             'Exports HQ', 'Exports PJM']

In [ ]:
# TODO move up?
generators_df = smooth_data(gen_rt_df.loc[start_date:end_date], smoothing_param)
generators_df.head()

In [ ]:
data_df[lbmp].plot(figsize=(10, 7))

## Feature selection

### Correlation

In [ ]:
plot_corr_plot(data_df.corr().round(2), figsize=(12, 8))

### Mutual info score

In [ ]:
from sklearn.feature_selection import mutual_info_regression

mi_scores = pd.DataFrame(columns=data_df.columns, index=data_df.columns)

# Calculate the mutual information scores
for i in range(len(data_df.columns)):
    for j in range(i, len(data_df.columns)):
        if i==j:
            continue
        mi_score = mutual_info_regression(data_df.iloc[:, i].values.reshape(-1, 1), data_df.iloc[:, j])
        mi_score = round(mi_score[0], 2)
        # mi_scores.iloc[i, j] = mi_score
        mi_scores.iloc[j, i] = mi_score
mi_scores = mi_scores.astype(float)

mi_scores

## Features forecasting

### Functions

In [ ]:
def fit_predict_prophet(params, train, test, y_name):
    '''
    train, test: pd.Series
    y_name: name of value to be forecasted, e.g. 'Close/Last'
    returns predicted value
    '''
    my_model = MyFbProphet([], params)
    my_model.fit(pd.DataFrame(train).drop(y_name, axis=1),
                 train)
    predicted = my_model.predict(pd.DataFrame(test).drop(y_name, axis=1))
    return predicted

In [ ]:
def do_train_valid_test_prophet(train, valid, train_valid, test, y_name):
    '''
    train, valid, train_valid, test: pd.Series with datetime index
    y_name: name of value to be forecasted, e.g. 'Close/Last'

    Returns tuple (mape_on_test and best_params)
    '''
    # validation step
    best_mape = float('inf')
    for param in get_param_grid(MyFbProphet([]).param_grid):
        predicted = fit_predict_prophet(param, train, valid, y_name)
        curr_mape = mape(valid.values, predicted)

        # print(my_model.name, ": ", curr_mape)
        # print('Params: ', param)

        if curr_mape <= best_mape:
            best_model = MyFbProphet([], param)
            best_mape = curr_mape
            best_params = param

    predicted = fit_predict_prophet(best_params, train_valid, test, y_name)

    curr_mape_test = mape(test.values, predicted)

    plot_actual_forecasted(test, predicted)

    return curr_mape_test, best_params


In [ ]:
def return_four_sets(dataset, y_name):
    '''
    dataset: pd.Series
    returns 4 sets of data: train, valid, train_valid, test
    '''
    train = dataset.iloc[:int(train_percentile * len(dataset))][y_name]
    valid = dataset.iloc[int(train_percentile * len(dataset)):int(valid_percentile * len(dataset))][y_name]

    train_valid = dataset.iloc[:int(valid_percentile * len(dataset))][y_name]
    test = dataset.iloc[int(valid_percentile * len(dataset)):][y_name]

    return train, valid, train_valid, test

### Gas

In [ ]:
four_sets = return_four_sets(gas_df, 'Close/Last')
gas_actual = four_sets[3]
gas_mape, gas_params = do_train_valid_test_prophet(four_sets[0], four_sets[1],
                                                   four_sets[2], four_sets[3], 'Close/Last')

In [ ]:
predicted = fit_predict_prophet(gas_params, four_sets[2], four_sets[3], 'Close/Last')
print(mape(predicted.values, four_sets[3].values))
plot_actual_forecasted(four_sets[3], predicted)

# TODO IMPORTANT TO RUN
gas_test_forecasted = predicted.set_axis(four_sets[3].index)
gas_test_forecasted

### Load

In [ ]:
name_of_load = 'Load'

load_series = pd.DataFrame(data_df[name_of_load])

four_sets = return_four_sets(load_series, name_of_load)
load_actual = four_sets[3]
load_mape, load_params = do_train_valid_test_prophet(four_sets[0], four_sets[1],
                                                     four_sets[2], four_sets[3], name_of_load)



In [ ]:
predicted = fit_predict_prophet(load_params, four_sets[2], four_sets[3], name_of_load)
print(mape(predicted.values, four_sets[3].values))
plot_actual_forecasted(four_sets[3], predicted)

# TODO IMPORTANT TO RUN
load_test_forecasted = predicted.set_axis(four_sets[3].index)
load_test_forecasted

### Plot

In [ ]:
rotation_angle = 45

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.plot(gas_actual, label='Actual Gas Price', color='blue')
ax1.plot(gas_test_forecasted, label='Predicted Gas Price', color='orange')

ax1.set_title('Actual vs Predicted Gas Price')
# ax1.set_xlabel('Time')
# ax1.set_ylabel('Gas')
ax1.tick_params(axis='x', rotation=rotation_angle)
ax1.set_ylim(0)
ax1.legend()

ax2.plot(load_actual, label='Actual Load', color='blue')
ax2.plot(load_test_forecasted, label='Predicted Load', color='orange')

ax2.set_title('Actual vs Predicted Load')
# ax2.set_xlabel('Time')
# ax2.set_ylabel('Load')
ax2.tick_params(axis='x', rotation=rotation_angle)
ax2.legend()

plt.tight_layout()

plt.show()

In [ ]:
fig.savefig('gas_load_temp.eps', format='eps')

## Overall training

In [ ]:
features = ['Load', 'Natural Gas', 'Dew temperature', 'Relative humidity']

In [ ]:
models = [
          # MyGradientBoosting,
          MyFbProphet,
          # MyRandomForest,
          # MyLinearRegression,
          # MyCNN,
          # MyLSTM
          ]

train_valid_len = int(len(data_df) * valid_percentile)
n_folds = 3
fold_size = int(train_valid_len/n_folds)
best_mape = float('inf')

gen_n = 0

for generator in generators_df.columns[375:460]:
    for m in models:
        best_mape = float('inf')
        for param in get_param_grid(m(features).param_grid):
            mape_scores = []
            for i in range(fold_size, train_valid_len+1, fold_size):
                train_data = data_df.iloc[:i]
                valid_data = data_df.iloc[i:i+fold_size]

                my_model = m(features, param)
                # my_model.fit(train_data.drop(lbmp, axis=1), train_data[lbmp])
                my_model.fit(train_data.drop(lbmp, axis=1),
                             generators_df[generator].loc[train_data.index])
                predicted = my_model.predict(valid_data.drop(lbmp, axis=1))
                # mape_scores.append(mape(valid_data[lbmp].values, predicted))
                mape_scores.append(mape(generators_df[generator].loc[valid_data.index].values,
                                        predicted))

            curr_mape = np.mean(mape_scores)
            # print(my_model.name, ": ", curr_mape)
            # print('Params: ', param)
            if curr_mape <= best_mape:
                best_model = my_model
                best_mape = curr_mape
                best_params = param

        best_model.save(generator + '|' + best_model.name, parameters=best_params)
    
    print('\n\n\n', gen_n, '\n\n\n')
    gen_n +=1    

In [ ]:
print(best_model)
print(best_mape)
print(best_params)

## Reading of saved models

In [ ]:
models_dict = {
               MyGradientBoosting(features).name: MyGradientBoosting,
               MyFbProphet(features).name: MyFbProphet,
               MyRandomForest(features).name: MyRandomForest,
               MyLinearRegression(features).name: MyLinearRegression,
               MyCNN(features).name: MyCNN,
              #  MyLSTM(features).name: MyLSTM
               }

In [ ]:
train_data = data_df.iloc[:int(len(data_df) * train_percentile)]
valid_data = data_df.iloc[int(len(data_df) * train_percentile):int(len(data_df) * valid_percentile)]
train_val_data = data_df.iloc[:int(len(data_df) * valid_percentile)]
test_data = data_df.iloc[int(len(data_df) * valid_percentile):]

In [ ]:
# updatind of test with forecasted values
test_data['Natural Gas'] = gas_test_forecasted
test_data['Load'] = load_test_forecasted
test_data = test_data.fillna(method='ffill')
test_data = test_data.fillna(method='bfill')

In [ ]:
path = '/content/gdrive/MyDrive/Diploma/Models/'
filenames = []
for entry in os.listdir(path):
        if os.path.isfile(os.path.join(path, entry)):
            filenames.append(entry)

res_dct = dict()
best_mape = float('inf')

for filename in filenames:
    # if not filename.startswith('|'):
    #     continue
    if filename.endswith('---cnn_model.pkl'):
        gen_name = filename[:-16]
        model_name = 'cnn_model'
    elif filename.endswith('_cnn_model.pkl'):
        gen_name = filename[:-14]
        model_name = 'cnn_model'
    elif filename.endswith('_prophet_model.pkl'):
        gen_name = filename[:-18]
        model_name = 'prophet_model'
    elif filename.endswith('_random_forest_model.pkl'):
        gen_name = filename[:-24]
        model_name = 'random_forest_model'
    else:
        gen_name = filename.split('|')[0]
        model_name = filename.split('|')[1].split('.')[0]

    # gen_name = filename.split('|')[0]
    # model_name = filename.split('|')[1].split('.')[0]


    model = models_dict[model_name](features)
    # model.read_params(filename.split('.')[0])
    model.read_params(filename[:-4])
    # model.fit(train_data.drop(lbmp, axis=1), train_data[lbmp])
    model.fit(train_data.drop(lbmp, axis=1),
              generators_df[gen_name].loc[train_data.index])
    predicted = model.predict(valid_data.drop(lbmp, axis=1))
    # curr_mape = mape(valid_data[lbmp].values, predicted)
    curr_mape = mape(generators_df[gen_name].loc[valid_data.index].values,
                     predicted)

    print(gen_name)
    print(model_name)
    print(curr_mape)
    print('--------------------------\n')

    try:
        res_dct[gen_name]
    except KeyError:
        res_dct[gen_name] = {
                             'best_mape': curr_mape,
                             'best_model': model_name
                            }
        continue
    if curr_mape < res_dct[gen_name]['best_mape']:
        res_dct[gen_name]['best_mape'] = curr_mape
        res_dct[gen_name]['best_model'] = model_name


In [ ]:
# Save dict??


with open('/content/gdrive/MyDrive/Diploma/res_dct.json', 'w') as f:
    json.dump(res_dct, f)

In [ ]:
with open('/content/gdrive/MyDrive/Diploma/res_dct.json', 'r') as f:
    res_dct = json.load(f)

In [ ]:
n = 1000
plt.figure(figsize=(10, 7))
plt.plot(valid_data.index[:n], valid_data[lbmp][:n], label='Actual')
plt.plot(valid_data.index[:n], predicted[:n], label='Forecasted')
plt.legend()

In [ ]:
df = pd.DataFrame.from_dict(res_dct, orient='index')

# because 200000
df = df.drop('SITHE___OGDNSBRG')
df.describe()#.boxplot()
# 'SITHE___OGDNSBRG'

In [ ]:
# # vals = ['mean', 'min', '25%', '50%', '75%', 'max']
# df[np.isclose(df['best_mape'],df.describe().loc['75%'], atol=0.01)].iloc[0].name

In [ ]:
df.describe().index

### Final strategy

In [ ]:
path = '/content/gdrive/MyDrive/Diploma/Models/'

final_df_predicted = pd.DataFrame(index=test_data.index)

for gen in res_dct.keys():
    mdl_name = res_dct[gen]['best_model']
    model = models_dict[mdl_name](features)
    try:
        model.read_params(gen + '|' + mdl_name)
    except FileNotFoundError:
        try:
            model.read_params(gen + '---' + mdl_name)
        except FileNotFoundError:
            model.read_params(gen + '_' + mdl_name)
    # model.fit(train_val_data.drop(lbmp, axis=1), train_val_data[lbmp])
    model.fit(train_val_data.drop(lbmp, axis=1),
              generators_df[gen].loc[train_val_data.index])
    predicted = model.predict(test_data.drop(lbmp, axis=1))
    if mdl_name=='prophet_model':
        predicted = predicted.set_axis(test_data.index)
    final_df_predicted[gen] = predicted
final_df_predicted

In [ ]:
def return_of_strategy(q, predicted_df, actual_df):
    list_accepted = []
    for i in range(len(predicted_df)):
        value = predicted_df.iloc[i].quantile(q)
        if value < actual_df.iloc[i].max():
            list_accepted.append(value)
    return np.sum(list_accepted)


In [ ]:
generators_df.head()

In [ ]:
lst = [i/100 for i in range(1, 101)]
res = []

for val in lst:
    res.append(return_of_strategy(val, final_df_predicted, generators_df.loc[test_data.index]))

plt.plot(lst, res)

In [ ]:
res[90:]

quantile 95 is the best - 2505

In [ ]:
np.max(res)

In [ ]:
res_gens = []
for gen in generators_df.columns:
    res_gens.append(generators_df.loc[test_data.index][gen].sum())

print(res_gens)
print(np.mean(res_gens))

In [ ]:
plt.plot(np.array(res_gens))

In [ ]:
plt.hist(np.array(res_gens), bins='auto', edgecolor='blue', label='Actual earnings \nof generators')

plt.xlabel('$/MWHr')
plt.ylabel('Number of generators')
# plt.title('Distribution of Values')

line_value = 2505
plt.axvline(x=line_value, color='r', linestyle='--', label='\nEarnings with proposed \nstrategy: {:.0f} $/MWHr'.format(line_value))

plt.legend(loc='upper left', bbox_to_anchor=(0, 1), fancybox=True, shadow=True)
plt.tight_layout()
plt.show()

In [ ]:
pd.DataFrame(res_gens).describe()

### Forecasting graph

In [ ]:
# axes[0][0]
vals = ['min', '25%', '50%', '75%', 'max', 'mean']
atol = 3 if q=='mean' else 0.01
df[np.isclose(df['best_mape'],df.describe().loc[vals[q]], atol=atol)].iloc[0].name


In [ ]:
vals = ['min', '25%', '50%', '75%', 'max', 'mean']

fig, axes = plt.subplots(3, 2, figsize=(8, 6), sharex=True)

for q in range(len(vals)):
    atol = 3 if vals[q]=='mean' else 0.01
    name_gen = df[np.isclose(df['best_mape'],df.describe().loc[vals[q]], atol=atol)].iloc[0].name
    actual = generators_df[name_gen].loc[final_df_predicted.index]
    predicted = final_df_predicted[name_gen]

    ax = axes[q//2][q%2]
    ax.plot(actual, label='Actual', color='blue')
    ax.plot(predicted, label='Predicted', color='orange')
    ax.set_title(vals[q])

    ax.tick_params(axis='x', rotation=45)
    # axes[q//2][q%2].legend()

fig.legend(['Actual value', 'Predicted value'], loc='upper center', ncol=2, bbox_to_anchor=(0.5, 1.05))

plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
rotation_angle = 45
# rows, cols
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.plot(gas_actual, label='Actual Gas Price', color='blue')
ax1.plot(gas_test_forecasted, label='Predicted Gas Price', color='orange')

ax1.set_title('Actual vs Predicted Gas Price')
# ax1.set_xlabel('Time')
# ax1.set_ylabel('Gas')
ax1.tick_params(axis='x', rotation=rotation_angle)
ax1.set_ylim(0)
ax1.legend()

ax2.plot(load_actual, label='Actual Load', color='blue')
ax2.plot(load_test_forecasted, label='Predicted Load', color='orange')

ax2.set_title('Actual vs Predicted Load')
# ax2.set_xlabel('Time')
# ax2.set_ylabel('Load')
ax2.tick_params(axis='x', rotation=rotation_angle)
ax2.legend()

plt.tight_layout()

plt.show()

In [ ]:
plt.plot(actual)
plt.plot(predicted)

## Permutation feature importance

In [ ]:
# valid_data = valid_data.drop(['Coal', 'Crude Oil', 'Uranium', 'Generation Scheduled', 'Wheel Throughs Bid',
#        'Gross Imports NYISO', 'Gross Exports HQ'], axis=1)
# train_data = train_data.drop(['Coal', 'Crude Oil', 'Uranium', 'Generation Scheduled', 'Wheel Throughs Bid',
#        'Gross Imports NYISO', 'Gross Exports HQ'], axis=1)

In [ ]:
valid_data_y = valid_data[lbmp]
valid_data = valid_data.drop(lbmp, axis=1)


In [ ]:
results = []

filename = 'best_for_choosen_gen_4H_4folds|gradient_boosting_model.pkl'
model_name = filename.split('|')[1].split('.')[0]
# model = models_dict[model_name](features)
# model.read_params(filename.split('.')[0])
model = MyFbProphet(features)

model.fit(train_data.drop([lbmp], axis=1), train_data[lbmp])
predicted = model.predict(valid_data)
curr_mape = mape(valid_data_y.values, predicted)
# print(curr_mape)

results.append({'feature': 'BASELINE','mape': curr_mape}) 

# for k in tqdm(range(len(COLS))):
for k in range(len(features)):
    
    # SHUFFLE FEATURE K
    save_col = valid_data.iloc[:, k].copy()
    np.random.shuffle(valid_data.iloc[:, k])
                        
    # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
    predicted = model.predict(valid_data)#.squeeze() 
    res_mape = mape(valid_data_y.values, predicted)
    results.append({'feature':features[k],'mape':res_mape})
    valid_data.iloc[:, k] = save_col

In [ ]:
df = pd.DataFrame(results)
df = df.sort_values('mape')
plt.figure(figsize=(4,7))
plt.barh(np.arange(len(features)+1),df.mape)
plt.yticks(np.arange(len(features)+1),df.feature.values)
# plt.title('Feature Importance',size=16)
plt.ylim((-1,len(features)+1))
plt.plot([curr_mape,curr_mape],[-1,len(features)+1], '--', color='orange',
                     label=f'Baseline\nMAPE={curr_mape:.3f}')
# plt.xlabel(f'Fold {fold+1} OOF MAE with feature permuted',size=14)
plt.ylabel('Feature',size=14)
plt.xlabel('MAPE',size=11)
plt.legend()
plt.show()

In [ ]:
df = pd.DataFrame(results)
df = df.sort_values('mape')
plt.figure(figsize=(8,4))
plt.bar(np.arange(len(features)+1),df.mape)
plt.xticks(np.arange(len(features)+1),df.feature.values, rotation=75)

# plt.title('Feature Importance',size=16)
plt.ylim((-1,40))
plt.plot([-1,len(features)+1],
         [curr_mape, curr_mape],
         '--', color='orange',
         label=f'Baseline\nMAPE={curr_mape:.3f}')

# plt.xlabel('Feature',size=14)
plt.ylabel('MAPE',size=11)
plt.legend()
plt.show()

## Lags analysis

In [ ]:
def create_lags_dataset(x, y, n_lags):
    temp_df = pd.DataFrame({"lag0": x})
    for cur_lag in range(1, n_lags + 1):
        temp_df["lag{}".format(cur_lag)] = x.shift(cur_lag)
    return temp_df.iloc[n_lags:], y.iloc[n_lags:]

In [ ]:
def calc_pacf(x, y):
    pacf = [x[x.columns[0]].corr(y)]
    for cur_lag in range(1, len(x.columns)):
        model = LinearRegression()
        cur_x = x[x.columns[:cur_lag]]
        model.fit(cur_x, y)
        residuals = y - model.predict(cur_x)
        pacf.append(x[x.columns[cur_lag]].corr(residuals))
    
    return np.array(pacf)

In [ ]:
def get_feature_importance_df(independent_columns, use_zero_lag_list, target_column, n_lags, feature_importance_fn):
    lags = list(range(n_lags + 1))
    feature_importances_dict = {"lag": lags}
    
    for column, use_zero_lag in zip(independent_columns, use_zero_lag_list):
        x, y = create_lags_dataset(column, target_column, n_lags)
        if not use_zero_lag:
            x.drop("lag0", axis=1, inplace=True)
        
        feature_importances = feature_importance_fn(x, y)
        if not use_zero_lag:
            feature_importances = np.concatenate((np.array([np.NaN]), feature_importances))
        
        feature_importances_dict[column.name] = feature_importances
    
    return pd.DataFrame(feature_importances_dict)

In [ ]:
columns = features
use_zero_lag_list = [True] * len(columns)
independent_columns = [data_df[column] for column in columns]
target_column = data_df[lbmp]

In [ ]:
n_lags = 100
pacf_importances = get_feature_importance_df(independent_columns, use_zero_lag_list, target_column, n_lags, calc_pacf)
pacf_importances.head(3)

In [ ]:
pacf_importances[columns].plot(figsize=(15, 8), legend=True, ylabel='PACF', xlabel='Lags')